In [ ]:
"""
This file is the entry point for the project
"""
# Begin by importing external libraries using path_setup
# NOTE : Must be ran first, thus PEP8-E402
import path_setup
path_setup.path_setup()
import math  # noqa E402
import sys  # noqa E402
import numpy as np # noqa E402
import datetime as dt # noqa E402
from LatticeClass_F import lattice_class as lt # noqa E402
from random import random  # noqa E402
import random as rnd  # noqa E402
# Shebang line for interactive output in vs_code, comment this out if you have troubles running the notebook
%matplotlib inline

def rand_time() -> int:
    out = int(dt.datetime.now().strftime('%s'))
    sys.stdout.write(f"Time Seed = {out}\n")
    return(int(dt.datetime.now().strftime('%s')))

def generate_random(gen_num: int) -> list:
    """
        Generates 2 or 3 random numbers whos sum is 100
    """
    if gen_num == 2:
        rand_a = rnd.randint(0, 100)
        rand_b = 100 - rand_a
        return([rand_a, rand_b])
    elif gen_num == 3:
        rand_a = rnd.randint(0, 98)
        if rand_a == 0:
            rand_b = rnd.randint(0, 99)
        else:
            rand_b = rnd.randint(0, 100-rand_a-1)
        rand_c = 100 - rand_a - rand_b
        return([rand_a, rand_b, rand_c])

# Setup
This is the setup for the simulation.

# Parameters
## N, M
> The span of the basis vectors

## $\beta J$'s
> The temperature

# Options 0 and 1
> 0 for seeded random or 1 for time based
## Seeded random : 0
> seeds random with 1644121893 by default to generate a repeatable test.

```Probs``` is a list containing 2 -- or 3 integers if voids is enabled, from 0 to 100 such that they all add together to 100.
Each entry in the ```Probs``` list represents the percent chance that random will assign a spin value of 1, -1, or
0 if lattice voids are enabled. You can play with these for interesting results. 
## Time based : 1
> seeds radom with the current epoch time as an integer.

# TODO
It shouldnt be hard to include external magnetic field interactions.

I want to make the $\beta J$ value distance dependant, but need to do more research to impliment this right now.

In [ ]:
# 32x32 runs in about 40-70 seconds depending on your system.
N = 48
M = 48
size = [N, M]
a = 0.1
b = 10
# The num_ponts settings increases computation time 
# dramatically, turn this down if you are just looking
# for good parameters to test. Turn up on a full data run.
# I suggest num_points < 100 for tests, and >= 1000 for full 
# data runs.
num_points = 100
step = (b-a)/num_points
BJs = np.arange(a, b, step)
total_time = math.trunc(np.sqrt(N*M))
BJ = 0.1

lt_c4v_up = lt(1, size)
lt_c3v_up = lt(1, size, [[1, 0], [0.5, np.sqrt(3)/2]])
lt_c6v_up = lt(1, size, [[0.5, np.sqrt(3)/2], [0.5, -np.sqrt(3)/2]])

lt_c4v_dn = lt(1, size)
lt_c3v_dn = lt(1, size, [[1, 0], [0.5, np.sqrt(3)/2]])
lt_c6v_dn = lt(1, size, [[0.5, np.sqrt(3)/2], [0.5, -np.sqrt(3)/2]])

output = input('Enter 0 for seeded random or 1 for time based:')
if output == '0':
    print("option 0 chosen..\n")
    # DOCtest seed = 1644121893
    seed = 1644121893
    lt_c4v_up.randomize(voids=True, probs=[15, 80, 5],
                        rand_seed=seed)
    lt_c3v_up.randomize(voids=True, probs=[15, 80, 5],
                        rand_seed=seed)
    lt_c6v_up.randomize(voids=True, probs=[15, 80, 5],
                        rand_seed=seed)
    lt_c4v_dn.randomize(voids=True, probs=[80, 15, 5],
                        rand_seed=seed)
    lt_c3v_dn.randomize(voids=True, probs=[80, 15, 5],
                        rand_seed=seed)
    lt_c6v_dn.randomize(voids=True, probs=[80, 15, 5],
                        rand_seed=seed)
else:
    print("option 1 chosen.\n")
    print("option 1 chosen.", end='\n')
    output = input('Enable voids (y/n)?')
    voids_enable = True if output == 'y' else False
    rand_n = 2 if voids_enable is False else 3
    seed = rand_time()

    lt_c4v_up.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c3v_up.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c6v_up.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c4v_dn.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c3v_dn.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c6v_dn.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)

# Display
Calling <lattice_object>.display() will display the current spin arangement of the lattice_object.

In [ ]:
lt_c4v_up.display()
lt_c3v_up.display()
lt_c6v_up.display()
lt_c4v_dn.display()
lt_c3v_dn.display()
lt_c6v_dn.display()

# Metropolis Algorithm test
Uncomment if you want to test these. The metropolis algorithm gets ran anyways in the next section.

In [ ]:
# Uncomment the next 4 lines below if you want, but not
# really a reason to as the metropolis algorithm gets
# called anyways from the get_spin_energy function.
# lt_a.metropolis(total_time, BJ, progress=True,
#                 save=False, auto_plot=True);
# lt_b.metropolis(total_time, BJ, progress=True,
#                 save=False, auto_plot=True);
# lt_c.metropolis(total_time, BJ, progress=True,
#                 save=False, auto_plot=True);

# Plotting Energy-Temperature relations
Can look at $\bar{m}$ as a function of temperature:
$$ T = 1/\beta k = J/(\beta J)k $$

We can also plot heat capacity as a function of time, using:

$$C_V = \sigma_E^2 / T^2 $$

$$= (\left<E^2\right>-\left<E\right>^2) \cdot \beta^{2} k^2$$

$$= \left(\left<\left(\frac{E}{J}\right)^2\right>-\left<\frac{E}{J}\right>^2 \right) \cdot (\beta J)^{2} k^2 $$

$$= \sigma_{E/J}^2 \cdot (\beta J)^{2} k^2$$

## Whats going on
The get_spin_genery function runs the Metropolis algorithm and finds a viable equlibrium temperature for a given $\beta J$ value, plotting 38
 runs of the metropolis algorithm chained together.

Then, we use the above relations to first plot $\hat{m}$ vs $\left(\frac{k}{J}\right)T$ -- where $\hat{m}$ the Sum of Spins over the number of spins.

Again using the above relations we can also plot $C_V / k^2$ vs $\left(\frac{k}{J}\right)T$.


In [ ]:
lt_c4v_up.get_spin_energy(BJs, total_time, save=False,
                          auto_plot=True);

In [ ]:
lt_c3v_up.get_spin_energy(BJs, total_time, save=False,
                          auto_plot=True);

In [ ]:
lt_c6v_up.get_spin_energy(BJs, total_time, save=False,
                          auto_plot=True);

In [ ]:
lt_c4v_dn.get_spin_energy(BJs, total_time, save=False,
                          auto_plot=True);

In [ ]:
lt_c3v_dn.get_spin_energy(BJs, total_time, save=False,
                          auto_plot=True);

In [ ]:
lt_c6v_dn.get_spin_energy(BJs, total_time, save=False,
                          auto_plot=True);